## Final Task

In [1]:
import pandas as pd
import numpy as np

In [2]:
application_train = pd.read_csv("home-credit-default-risk/application_train.csv")
application_test = pd.read_csv("home-credit-default-risk/application_test.csv")

application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
columns_list = list(application_test.columns)
print(columns_list)

['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG

In [5]:
bureau = pd.read_csv("home-credit-default-risk/bureau.csv")
bureau_balance = pd.read_csv("home-credit-default-risk/bureau_balance.csv")
previous_application = pd.read_csv("home-credit-default-risk/previous_application.csv")
pos_cash_balance = pd.read_csv("home-credit-default-risk/POS_CASH_balance.csv")
installments_payments = pd.read_csv("home-credit-default-risk/installments_payments.csv")
credit_card_balance = pd.read_csv("home-credit-default-risk/credit_card_balance.csv")

In [7]:
# Aggregate bureau_balance untuk mendapatkan informasi ringkas
bureau_balance_agg = bureau_balance.groupby("SK_ID_BUREAU").agg({'MONTHS_BALANCE': ['min', 'max', 'count']})

# Flatten multi-index columns
bureau_balance_agg.columns = ['_'.join(col) for col in bureau_balance_agg.columns]
bureau_balance_agg.reset_index(inplace=True)

# Merge dengan bureau
bureau_merged = bureau.merge(bureau_balance_agg, on="SK_ID_BUREAU", how="left")

In [8]:
# Aggregate bureau untuk mendapatkan informasi ringkas per SK_ID_CURR
bureau_agg = bureau_merged.groupby("SK_ID_CURR").agg({'SK_ID_BUREAU': 'count'}).reset_index()
bureau_agg.rename(columns={'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT'}, inplace=True)

# Merge dengan application_train
application_train = application_train.merge(bureau_agg, on="SK_ID_CURR", how="left")
application_test = application_test.merge(bureau_agg, on="SK_ID_CURR", how="left")

In [10]:
# Aggregate POS_CASH_balance
pos_cash_agg = pos_cash_balance.groupby("SK_ID_PREV").agg({'MONTHS_BALANCE': ['min', 'max', 'count']})
pos_cash_agg.columns = ['_'.join(col) for col in pos_cash_agg.columns]  # Flatten multi-index
pos_cash_agg.reset_index(inplace=True)

# Aggregate installments_payments
installments_agg = installments_payments.groupby("SK_ID_PREV").agg({'NUM_INSTALMENT_VERSION': 'nunique'})
installments_agg.columns = ['_'.join(col) for col in installments_agg.columns]
installments_agg.reset_index(inplace=True)

# Aggregate credit_card_balance
credit_card_agg = credit_card_balance.groupby("SK_ID_PREV").agg({'MONTHS_BALANCE': ['min', 'max', 'count']})
credit_card_agg.columns = ['_'.join(col) for col in credit_card_agg.columns]
credit_card_agg.reset_index(inplace=True)

# Merge semua dengan previous_application
previous_application = previous_application.merge(pos_cash_agg, on="SK_ID_PREV", how="left")
previous_application = previous_application.merge(installments_agg, on="SK_ID_PREV", how="left")
previous_application = previous_application.merge(credit_card_agg, on="SK_ID_PREV", how="left")

In [11]:
# Aggregate previous_application
previous_application_agg = previous_application.groupby("SK_ID_CURR").agg({'SK_ID_PREV': 'count'}).reset_index()
previous_application_agg.rename(columns={'SK_ID_PREV': 'PREVIOUS_LOAN_COUNT'}, inplace=True)

# Merge dengan application_train
application_train = application_train.merge(previous_application_agg, on="SK_ID_CURR", how="left")
application_test = application_test.merge(previous_application_agg, on="SK_ID_CURR", how="left")


In [12]:
application_train.to_csv("application_train_merged.csv", index=False)
application_test.to_csv("application_test_merged.csv", index=False)